In [1]:
# Import necessary libraries
from gnews import GNews
import pandas as pd
import requests
from fake_useragent import UserAgent
import newspaper
from newspaper import fulltext
import re


In [2]:
# Define the keyword to search.
keyword = 'discapacidad'

In [3]:
# Perform news scraping from Google and extract the result into Pandas dataframe. 
google_news = GNews(language='es', country='AR', period='7d', max_results=100)

news_result = google_news.get_news(keyword)

# Convert to dataframe
news_data_df = pd.DataFrame(news_result)

# Show sample
news_data_df.head()


,title,description,published date,url,publisher
0,Emergencia en Discapacidad: el Gobierno la pro...,Emergencia en Discapacidad: el Gobierno la pro...,"Tue, 16 Sep 2025 03:01:00 GMT",https://news.google.com/rss/articles/CBMimwFBV...,"{'href': 'https://www.pagina12.com.ar', 'title..."
1,El Gobierno confirmó que promulgará la ley de ...,El Gobierno confirmó que promulgará la ley de ...,"Mon, 15 Sep 2025 12:03:09 GMT",https://news.google.com/rss/articles/CBMi6wFBV...,"{'href': 'https://www.infobae.com', 'title': '..."
2,INTEGRANTES DEL MPD PARTICIPARON DE UN ENCUENT...,INTEGRANTES DEL MPD PARTICIPARON DE UN ENCUENT...,"Tue, 16 Sep 2025 15:28:51 GMT",https://news.google.com/rss/articles/CBMiuwFBV...,"{'href': 'https://www.mpd.gov.ar', 'title': 'M..."
3,Andy Kusnetzoff rompió en llanto al hablar de ...,Andy Kusnetzoff rompió en llanto al hablar de ...,"Wed, 17 Sep 2025 19:17:00 GMT",https://news.google.com/rss/articles/CBMi8AFBV...,"{'href': 'https://www.lanacion.com.ar', 'title..."
4,Pensión por discapacidad ANSES: cuánto se cobr...,Pensión por discapacidad ANSES: cuánto se cobr...,"Sat, 13 Sep 2025 17:10:17 GMT",https://news.google.com/rss/articles/CBMivwFBV...,"{'href': 'https://www.clarin.com', 'title': 'C..."


In [ ]:
from gnews import GNews
import pandas as pd
import time

def search_news(keywords, countries, languages='en', period='7d', max_results=50):
    """Quick news search across countries and languages"""
    keywords = [keywords] if isinstance(keywords, str) else keywords
    countries = [countries] if isinstance(countries, str) else countries
    languages = [languages] if isinstance(languages, str) else languages
    
    articles = []
    for kw in keywords:
        for c in countries:
            for l in languages:
                try:
                    news = GNews(language=l, country=c, period=period, max_results=max_results)
                    results = news.get_news(kw)
                    for r in results:
                        r.update({'keyword': kw, 'country': c, 'language': l})
                    articles.extend(results)
                    time.sleep(1)
                except:
                    continue
    
    return pd.DataFrame(articles)

common_countries = [
        'US', 'GB', 'CA', 'AU', 'IN', 'DE', 'FR', 'IT', 'ES', 'BR',
        'MX', 'AR', 'CL', 'CO', 'PE', 'CN', 'JP', 'KR', 'RU', 'ZA',
        'NG', 'EG', 'KE', 'SA', 'AE', 'TR', 'NL', 'SE', 'NO', 'DK',
        'FI', 'BE', 'CH', 'AT', 'PT', 'GR', 'IL', 'SG', 'MY', 'ID',
        'PH', 'VN', 'TH', 'NZ', 'IE'
    ]

df = search_news(keywords=['discapacidad','natalidad'], countries=['AR', 'US'], languages=['en','es'],period='7d', max_results= 50)

df

09/18/2025 02:43:53 PM - HTTPSConnectionPool(host='news.google.com', port=443): Max retries exceeded with url: /rss/articles/CBMitgFBVV95cUxPN3h1WUhrOEk3MFNhSFFVN216S2lVSmEzQzJhMWpnMnFwY0JkX1VaMERLNVk1bVpEVS00RXlRUTVVRlZTN0x4S3JqQ3NIMThMWVhjaHEyUjkwYWN3YkNvNGZhNzIzdjFBRDdMcV9PbGliUW4zc0FpcGtlemlVWFZhWk5hVmJ1NWI0ZGttQ0oteUVmdVdIQkdpdjU0dm5hbER1NS11UmxJX0dfZGpIYnlmYVYwT3pRUdIBuwFBVV95cUxOSTNOY3UtTkFnNmE1MnRyV3EtTlg1UU9JWC1lYXp6b2ZWZmhnWlNKVHpWdGlIU2s5MFlscXQ3MkZrT3NaYUJ6RHVqT3dmazdVd1VoRVNkMk80V21uYk5DYjlJczdiWm5lUlJVdFBwUnFkQ3UzN1VaYTFMTTlQWEZKamZUaVRtNVBLZzlDN2RzMFh1dzVsTHAta3hlWk9NaHdjS3Rmd1d6REhNY3NPWnRTcFBfMmZhSlNLMjM0?oc=5 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))


In [31]:
# from newspaper import Article
# ua = UserAgent()
# news_data_df_with_text = []

# for index, row in news_data_df.iterrows():
#     news_title = str(row.get('title', ''))
#     news_media = str(row.get('publisher', {}).get('title', ''))
#     news_update = str(row.get('published date', ''))
#     news_link = str(row.get('url', ''))
#     news_description = str(row.get('description', ''))

#     print(news_link)

#     text = ""  # initialize

#     # First try Newspaper (more robust)
#     try:
#         article = Article(news_link, browser_user_agent=ua.chrome)
#         article.download()
#         article.parse()
#         text = article.text
#         print('Text Content via newspaper3k')
#     except Exception as e:
#         print(f'Newspaper extraction failed: {e}')
#         # fallback to requests + fulltext
#         try:
#             html = requests.get(news_link, headers={'User-Agent': ua.chrome}, timeout=5).text
#             text = fulltext(html)
#             print('Text Content via fulltext fallback')
#         except Exception as e2:
#             print(f'Fallback fulltext failed: {e2}')
#             # text remains empty

#     news_data_df_with_text.append([
#         news_title, news_media, news_update,
#         news_description, news_link, text
#     ])

# news_data_with_text_df = pd.DataFrame(
#     news_data_df_with_text,
#     columns=['Title', 'Media', 'Published Date',
#              'Description', 'Link', 'Text']
# )


In [ ]:
from mailjet_rest import Client
import pandas as pd

# Build HTML content for the email
def build_news_html(df, max_chars=500):
    html = f"<h3>Noticias de hoy </h3>"
    for _, row in df.iterrows():
        html += f"<h4>{row['title']}</h4>"
        html += f"<p><strong>Descripción:</strong> {row['description']}</p>"
        html += f"<p><a href='{row['url']}'>Leer artículo completo</a></p><hr>"
    return html


news_html = build_news_html(news_data_df)

# Mailjet setup
api_key = '15c19282a90f7c1e5aec38ae3653a2e3'
api_secret = 'a2db6c0dd8541c30df707c6a7350d926'

mailjet = Client(auth=(api_key, api_secret), version='v3.1')

data = {
  'Messages': [
    {
      "From": {
        "Email": "anapaulacastillozuain@gmail.com",
        "Name": "Ana"
      },
      "To": [
        {
          "Email": "anapaulacastillozuain@gmail.com",
          "Name": "Ana"
        }
      ],
      "Subject": f"Noticias de {keyword}",
      "HTMLPart": news_html
    }
  ]
}

# Send the email
result = mailjet.send.create(data=data)
print(result.status_code)
print(result.json())

200
{'Messages': [{'Status': 'success', 'CustomID': '', 'To': [{'Email': 'anapaulacastillozuain@gmail.com', 'MessageUUID': '6458b1dc-e174-4a3b-abaf-a4697cdeae95', 'MessageID': 576460784718253331, 'MessageHref': 'https://api.mailjet.com/v3/REST/message/576460784718253331'}], 'Cc': [], 'Bcc': []}]}


In [ ]:
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# import time
# import re

# def extract_final_urls_selenium(urls, headless=True, delay=3):
#     """
#     Extract final URLs from Google News redirect links using Selenium
    
#     Args:
#         urls (list): List of Google News redirect URLs
#         headless (bool): Run browser in background
#         delay (int): Additional wait time in seconds
    
#     Returns:
#         list: Final URLs
#     """
#     chrome_options = Options()
#     if headless:
#         chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--disable-gpu")
#     chrome_options.add_argument("--no-sandbox")
#     chrome_options.add_argument("--disable-dev-shm-usage")
    
#     driver = webdriver.Chrome(options=chrome_options)
#     final_urls = []
    
#     try:
#         for i, url in enumerate(urls):
#             try:
#                 # Ensure URL has https://
#                 if not url.startswith(('http://', 'https://')):
#                     url = 'https://' + url
                
#                 print(f"Processing URL {i+1}/{len(urls)}: {url}")
                
#                 driver.get(url)
                
#                 # Wait for page to load
#                 WebDriverWait(driver, 10).until(
#                     lambda d: d.execute_script('return document.readyState') == 'complete'
#                 )
                
#                 # Additional delay
#                 time.sleep(delay)
                
#                 final_url = driver.current_url
#                 final_urls.append(final_url)
                
#                 print(f"  → Final URL: {final_url}")
                
#             except Exception as e:
#                 print(f"  → Error processing URL {i+1}: {e}")
#                 final_urls.append(f"ERROR: {str(e)}")
                
#     finally:
#         driver.quit()
    
#     return final_urls

# # Example usage with a DataFrame
# def process_dataset_selenium(df, url_column='url', output_file='processed_data.csv'):
#     """
#     Process a DataFrame containing Google News redirect URLs
    
#     Args:
#         df: pandas DataFrame
#         url_column: name of the column containing URLs
#         output_file: name of the output CSV file
#     """
#     # Get URLs from the specified column
#     urls = df[url_column].tolist()
    
#     # Extract final URLs
#     final_urls = extract_final_urls_selenium(urls)
    
#     # Add final URLs to DataFrame
#     df['final_url'] = final_urls
    
#     # Save results
#     df.to_csv(output_file, index=False)
#     print(f"Results saved to {output_file}")
    
#     return df


# processed_df = process_dataset_selenium(news, url_column='url')

Processing URL 1/5: https://news.google.com/rss/articles/CBMi6wFBVV95cUxQQ2Y3dnY0dU5MeUpFaGJmOGhUMTJCMklpbnZ4VHNqTVhhUzZsZzNRbFNLVmVBMVE3OXNna2ZINGtKNkxDS1dSeWpfWkxLaENFWXNINFh1cFJPbVhnaXR4cjNxVk5UZ0oyZ0tfTmpvLTZWdE5lLUhFbEozeG5MeFN4b2RhRkFJaUpUVVZFOWF3M3VqY1d3X1hSMHBGM3ZyamRlUzZGdFZnWGtUMTJmb2RfNS1YT1ZVRXNGV3dMMElWc3RpZjJSTWpoam5fS2FYa0pxSllfVUVLZU9uY005MUMzdGI5WWZVZ2lmR05F0gGGAkFVX3lxTE1tQ3JQU0ozcnpWc01fSW16RDkxbjd4Vkl1S3l3TlRMQVlVb1F1SmFLdlZaeWhZTVEwV08zcGplUXBZMEtsTXFfWUxDU05QWW11d2FuS3kwZWZYSkxaZ2NORWtpaDd3NzBUOVU0ZnE2RHlXVzVkbS14SHdBb3ZjSmNOOGJzUVA2dFYwOF9HdXp3V0ItVDhIRlU4ektzMlpYN3AtWVlkLXFtc3ZiamVJSEJsa1lETkRrTmd1Yk5JNUdOSlRNdmlmeU9xdEg3SWNXSkZONi1RdWdSOTV1ZDlNZVpCSDVHOXg5RTFpWGtWa21fSk56VnRDWXJBbTFWa3NFWDFCbmlIUmc?oc=5&hl=en-US&gl=US&ceid=US:en
  → Final URL: https://www.infobae.com/politica/2025/09/15/el-gobierno-confirmo-que-promulgara-la-ley-de-discapacidad-pero-no-la-aplicara-hasta-definir-el-financiamiento/
Processing URL 2/5: https://news.google.com/rss/articles/CBMimAFB

09/15/2025 03:49:54 PM - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPConnectionPool(host='localhost', port=63258): Read timed out. (read timeout=120)")': /session/5908853a913642e471f2e564ff46eb8b/url


In [ ]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time

# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in background
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox")

# initial_url = "https://news.google.com/rss/articles/CBMixwFBVV95cUxORklIOXFnYng0aHQ1WldVM0cwRUVfWHA2aWJJbzBhWUhCcWZOUXhJSnNlVlBtZ3RIMmVkSktiZU5FN2xUeWRKNnFkT2hwSnBWRmg4bmpZSXMxSEFMMzZTeFdLQUFsUl9WRk1QYlg4OV9US3FVa3BzdlE3ODFMZWF4eW9UeDZKR2Q2ck9vT0h0eFVtUTBUbzYyYzZjS1pJUGY2eGVGRDhSRHVwUWRGa2NDN0tsMU9JbVVlQnJIajV5d1FZUkd5aTZr?oc=5&hl=en-US&gl=US&ceid=US:en"

# driver = webdriver.Chrome(options=chrome_options)

# try:
#     driver.get(initial_url)
    
#     # Wait for page to load completely
#     WebDriverWait(driver, 10).until(
#         lambda d: d.execute_script('return document.readyState') == 'complete'
#     )
    
#     # Additional waiting time if needed
#     time.sleep(3)
    
#     final_url = driver.current_url
#     print(f"The final URL is: {final_url}")
    
# finally:
#     driver.quit()

The final URL is: https://www.lanacion.com.ar/politica/un-juez-ordeno-que-se-restablezcan-las-pensiones-por-discapacidad-en-catamarca-nid15092025/
